In [1]:
import pandas as pd

In [2]:
df_elo=pd.read_csv(r"C:\Users\jacob\Desktop\nbaallelo\nbaallelo.csv")

In [3]:
df_elo.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN


In [4]:
df_elo.shape

(126314, 23)

In [6]:
df_elo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126314 entries, 0 to 126313
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   gameorder      126314 non-null  int64  
 1   game_id        126314 non-null  object 
 2   lg_id          126314 non-null  object 
 3   _iscopy        126314 non-null  int64  
 4   year_id        126314 non-null  int64  
 5   date_game      126314 non-null  object 
 6   seasongame     126314 non-null  int64  
 7   is_playoffs    126314 non-null  int64  
 8   team_id        126314 non-null  object 
 9   fran_id        126314 non-null  object 
 10  pts            126314 non-null  int64  
 11  elo_i          126314 non-null  float64
 12  elo_n          126314 non-null  float64
 13  win_equiv      126314 non-null  float64
 14  opp_id         126314 non-null  object 
 15  opp_fran       126314 non-null  object 
 16  opp_pts        126314 non-null  int64  
 17  opp_elo_i      126314 non-nul

In [10]:
df_elo['year_id'].count

<bound method Series.count of 0         1947
1         1947
2         1947
3         1947
4         1947
          ... 
126309    2015
126310    2015
126311    2015
126312    2015
126313    2015
Name: year_id, Length: 126314, dtype: int64>

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Encode categorical variables in df_elo
encoder = LabelEncoder()
df_elo['game_location'] = encoder.fit_transform(df_elo['game_location'])
df_elo['game_result'] = encoder.fit_transform(df_elo['game_result'])  # 1 for win, 0 for loss

# Select features and target from df_elo
X_elo = df_elo[['elo_i', 'opp_elo_i', 'game_location', 'is_playoffs']]
y_elo = df_elo['game_result']

# Split df_elo into training and testing sets
X_elo_train, X_elo_test, y_elo_train, y_elo_test = train_test_split(X_elo, y_elo, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Classifier on df_elo
model_elo = RandomForestClassifier(n_estimators=100, random_state=42)
model_elo.fit(X_elo_train, y_elo_train)

# Predict on the df_elo testing set
y_elo_pred = model_elo.predict(X_elo_test)

# Evaluate the model with df_elo data
accuracy_elo = accuracy_score(y_elo_test, y_elo_pred)
print(accuracy_elo)


0.6325456200767922


In [19]:
Adding a simple team performance metric - average points per game
df_elo['avg_pts'] = df_elo.groupby('team_id')['pts'].transform(lambda x: x.expanding().mean())


In [20]:
# Encode categorical variables in df_elo
encoder = LabelEncoder()
df_elo['game_location'] = encoder.fit_transform(df_elo['game_location'])
df_elo['game_result'] = encoder.fit_transform(df_elo['game_result'])  # 1 for win, 0 for loss

# Select features and target from df_elo
X_elo = df_elo[['elo_i', 'opp_elo_i', 'game_location', 'is_playoffs','avg_pts']]
y_elo = df_elo['game_result']

# Split df_elo into training and testing sets
X_elo_train, X_elo_test, y_elo_train, y_elo_test = train_test_split(X_elo, y_elo, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Classifier on df_elo
model_elo = RandomForestClassifier(n_estimators=100, random_state=42)
model_elo.fit(X_elo_train, y_elo_train)

# Predict on the df_elo testing set
y_elo_pred = model_elo.predict(X_elo_test)

# Evaluate the model with df_elo data
accuracy_elo = accuracy_score(y_elo_test, y_elo_pred)
print(accuracy_elo)



0.6500415627597672


In [21]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_elo_train, y_elo_train)

best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_elo_test)
best_accuracy = accuracy_score(y_elo_test, y_pred_best)
print(f"Best Model accuracy: {best_accuracy}")


Best Model accuracy: 0.6782250722400348
